<a href="https://colab.research.google.com/github/HazemmoAlsady/AWN_Graduation_Project/blob/main/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U \
langchain \
langchain-community \
langchain-text-splitters \
faiss-cpu \
sentence-transformers \
transformers \
accelerate


In [3]:
import os

os.makedirs("docs", exist_ok=True)


In [4]:
with open("docs/register.md", "w", encoding="utf-8") as f:
    f.write("""
# التسجيل في منصة عون

## خطوات التسجيل
- الدخول على موقع عون
- إنشاء حساب جديد
- إدخال البيانات الأساسية
- تأكيد رقم الهاتف

## ملاحظات
- يجب إدخال بيانات صحيحة
- رقم الهاتف إلزامي
""")


In [5]:
with open("docs/case_submit.md", "w", encoding="utf-8") as f:
    f.write("""
# رفع حالة في منصة عون

## خطوات رفع الحالة
- تسجيل الدخول إلى الحساب
- اختيار رفع حالة جديدة
- إدخال بيانات الأسرة
- إرفاق المستندات المطلوبة
- إرسال الطلب للمراجعة

## ملاحظات
- يجب إرفاق مستندات واضحة
- الطلب يخضع للمراجعة
""")


In [6]:
with open("docs/terms.md", "w", encoding="utf-8") as f:
    f.write("""
# شروط الاستفادة من منصة عون

- أن تكون البيانات صحيحة
- أن تكون الحالة حقيقية
- تقديم المستندات المطلوبة
- قبول سياسة الاستخدام
""")


In [7]:
with open("docs/faq.md", "w", encoding="utf-8") as f:
    f.write("""
# الأسئلة الشائعة

## هل التسجيل مجاني؟
نعم، التسجيل مجاني تمامًا.

## هل يمكن تعديل البيانات؟
نعم، يمكن تعديل البيانات قبل إرسال الطلب.
""")


In [8]:
!ls docs


case_submit.md	faq.md	register.md  terms.md


In [9]:
from langchain_community.document_loaders import TextLoader

files = [
    "docs/register.md",
    "docs/case_submit.md",
    "docs/terms.md",
    "docs/faq.md"
]

documents = []

for file in files:
    loader = TextLoader(file, encoding="utf-8")
    documents.extend(loader.load())

print("عدد الملفات:", len(documents))


عدد الملفات: 4


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

chunks = splitter.split_documents(documents)

print("عدد القطع:", len(chunks))


عدد القطع: 4


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

vector_db = FAISS.from_documents(chunks, embeddings)


/tmp/ipython-input-3538284932.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline(
    task="text-generation",
    model="google/flan-t5-base",
    max_new_tokens=200
)

llm = HuggingFacePipeline(pipeline=pipe)


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaF

In [15]:
from langchain.chains.retrieval_qa.base import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(),
    chain_type="stuff"
)


ModuleNotFoundError: No module named 'langchain.chains'

In [ ]:
print(qa.run("أسجل في عون إزاي؟"))


In [ ]:
print(qa.run("إزاي أرفع حالة؟"))
print(qa.run("هل في مساعدات طبية؟"))
